In [1]:
from dataclasses import dataclass
from typing import Optional
import random

### Simple Routing with discrete congestion cost pricing 

In [4]:
# Two routes
routes = {
    'S98': {'cost': 7, 'capacity': 5, 'trucks': 0},
    'I25': {'cost': 8, 'capacity': 5, 'trucks': 0}
}

# 30 trucks
total_cost = 0

for truck_num in range(1, 20):
    
    # Try each route
    options = []
    for name, route in routes.items():
        cost = route['cost']
        
        # Congestion penalties
        if route['trucks'] >= route['capacity'] * 4:
            cost += 50  
            status = "Critical"
        elif route['trucks'] >= route['capacity']*2:
            cost += 25  
            status = "FULL"
        elif route['trucks'] > route['capacity']:
            cost += 10  # Regular congestion penalty
            status = "Congested"
        elif route['trucks'] <= route['capacity']:
            cost += 0  # Regular congestion penalty
            status = "At/ less than Capacity"
        else:
            status = f"{route['trucks']}/{route['capacity']}"
        
        options.append({
            'name': name,
            'route': route,
            'cost': cost,
            'status': status
        })
        print(f"  {name}: ${cost:3d} ({status})")
    
    # Pick cheapest
    best = min(options, key=lambda x: x['cost'])
    best['route']['trucks'] += 1
    total_cost += best['cost']
    
    print(f"ASSIGN truck {truck_num}: {best['name']} route")

for name, route in routes.items():
    pct = (route['trucks'] / 15) * 100
    over_capacity = route['trucks'] - route['capacity']
    if route['trucks'] >= route['capacity'] * 2:
        status = f"CRITICAL ({over_capacity} over)"
    elif route['trucks'] >= route['capacity']:
        status = f"FULL ({over_capacity} over)"
    else:
        status = "OK"
    print(f"{name:5s}: {route['trucks']} trucks ({pct:.0f}%) - {status}")
print(f"\nTotal cost: ${total_cost}")
print(f"Average cost per truck: ${total_cost/30:.2f}")

  S98: $  7 (At/ less than Capacity)
  I25: $  8 (At/ less than Capacity)
ASSIGN truck 1: S98 route
  S98: $  7 (At/ less than Capacity)
  I25: $  8 (At/ less than Capacity)
ASSIGN truck 2: S98 route
  S98: $  7 (At/ less than Capacity)
  I25: $  8 (At/ less than Capacity)
ASSIGN truck 3: S98 route
  S98: $  7 (At/ less than Capacity)
  I25: $  8 (At/ less than Capacity)
ASSIGN truck 4: S98 route
  S98: $  7 (At/ less than Capacity)
  I25: $  8 (At/ less than Capacity)
ASSIGN truck 5: S98 route
  S98: $  7 (At/ less than Capacity)
  I25: $  8 (At/ less than Capacity)
ASSIGN truck 6: S98 route
  S98: $ 17 (Congested)
  I25: $  8 (At/ less than Capacity)
ASSIGN truck 7: I25 route
  S98: $ 17 (Congested)
  I25: $  8 (At/ less than Capacity)
ASSIGN truck 8: I25 route
  S98: $ 17 (Congested)
  I25: $  8 (At/ less than Capacity)
ASSIGN truck 9: I25 route
  S98: $ 17 (Congested)
  I25: $  8 (At/ less than Capacity)
ASSIGN truck 10: I25 route
  S98: $ 17 (Congested)
  I25: $  8 (At/ less than 

### Step 1: Simple Freeze and Assigning with discrete congestion cost pricing, trucks approaching from random distances and times

In [8]:
##### from dataclasses import dataclass
from typing import Optional
import random

#Create truck data class
@dataclass
class Truck:
    id: int
    name: str
    position: float  # Miles from ABQ
    speed: float     # MPH
    route: Optional[str] = None
    frozen: bool = False
    freeze_time: float = 0
    actual_cost: float = 0 


#Number of trucks? randomly 25 to 50 miles out and random speed between 50 and 70mph
def create_trucks(num_trucks):
    trucks = []
    for i in range(1, num_trucks + 1):
        truck = Truck(
            id=i,
            name=f"Truck {i}",
            position=random.randint(25, 50),  
            speed=random.randint(50, 70)     
        )
        trucks.append(truck)
    return trucks

#Time to ABQ 
def calculate_time_to_abq(truck):
    if truck.position <= 0:
        return 0
    return (truck.position / truck.speed) * 60 #in mins

def assign_route(truck, routes):
    S98 = routes['S98']
    I25 = routes['I25']
    
    #  cost for S98 route
    S98_cost = S98['cost']
    if S98['count'] >= S98['capacity']:
        S98_cost += 50  # Full
    elif S98['count'] >= S98['capacity'] * 0.8:
        S98_cost += 25   # Congested (80% full)
        
    #  cost for I25 route
    I25_cost = I25['cost']
    if I25['count'] >= I25['capacity']:
        I25_cost += 50  # Full - big penalty
    elif I25['count'] >= I25['capacity'] * 0.8:
        I25_cost += 25   # 80% full - medium penalty
    
    # Pick cheapest route
    if S98_cost <= I25_cost:
        truck.route = 'S98'
        S98['count'] += 1
        cost = S98_cost
    else:
        truck.route = 'I25'
        I25['count'] += 1
        cost = I25_cost
    
    return cost

#Interactive
num_trucks = int(input("\nHow many trucks? "))
trucks = create_trucks(num_trucks)

routes = {
    'S98': {'cost': 7, 'capacity': 5, 'count': 0},
    'I25': {'cost': 8, 'capacity': 5, 'count': 0}
}

#set freeze 15 mins from ABQ
freeze_horizon = 15

# initial positions
print("Initial positions:")
for truck in sorted(trucks, key=lambda t: t.position, reverse=True):
    time_to_abq = calculate_time_to_abq(truck)
    print(f"{truck.name}: {truck.position:.1f} mi, {truck.speed} mph & {time_to_abq:.1f} min to ABQ")


How many trucks?  30


Initial positions:
Truck 8: 47.0 mi, 69 mph & 40.9 min to ABQ
Truck 7: 46.0 mi, 66 mph & 41.8 min to ABQ
Truck 5: 45.0 mi, 66 mph & 40.9 min to ABQ
Truck 15: 45.0 mi, 53 mph & 50.9 min to ABQ
Truck 28: 45.0 mi, 59 mph & 45.8 min to ABQ
Truck 10: 42.0 mi, 66 mph & 38.2 min to ABQ
Truck 6: 41.0 mi, 61 mph & 40.3 min to ABQ
Truck 11: 41.0 mi, 59 mph & 41.7 min to ABQ
Truck 18: 40.0 mi, 68 mph & 35.3 min to ABQ
Truck 30: 40.0 mi, 68 mph & 35.3 min to ABQ
Truck 14: 38.0 mi, 68 mph & 33.5 min to ABQ
Truck 21: 38.0 mi, 69 mph & 33.0 min to ABQ
Truck 9: 37.0 mi, 67 mph & 33.1 min to ABQ
Truck 13: 36.0 mi, 53 mph & 40.8 min to ABQ
Truck 25: 36.0 mi, 59 mph & 36.6 min to ABQ
Truck 26: 33.0 mi, 70 mph & 28.3 min to ABQ
Truck 29: 33.0 mi, 51 mph & 38.8 min to ABQ
Truck 1: 32.0 mi, 50 mph & 38.4 min to ABQ
Truck 4: 30.0 mi, 68 mph & 26.5 min to ABQ
Truck 16: 30.0 mi, 52 mph & 34.6 min to ABQ
Truck 23: 30.0 mi, 51 mph & 35.3 min to ABQ
Truck 20: 29.0 mi, 63 mph & 27.6 min to ABQ
Truck 24: 29.0 mi, 6

In [10]:
# Main loop
time = 0
while any(not t.frozen for t in trucks):
    
    print(f"\n Time: {time} minutes")

    #Check for trucks at freeze horizon
    frozen_this_cycle=[]
    for truck in trucks:
        if not truck.frozen: 
            time_to_abq = calculate_time_to_abq(truck)

            if 0 < time_to_abq <= freeze_horizon:
                frozen_this_cycle.append((truck, time_to_abq))
    
    if frozen_this_cycle:
        print("Freeze Horizon Crossed")
        for truck, time_to_abq in frozen_this_cycle:
                    cost = assign_route(truck, routes)
                    truck.frozen = True
                    truck.freeze_time = time
                    truck.actual_cost = cost
                    
                    print(f"  {truck.name}:")
                    print(f"    Position: {truck.position:.1f} mi")
                    print(f"    Time to ABQ: {time_to_abq:.1f} min: Within 15 min")
                    print(f"    Assigned to {truck.route} route (${cost})")
    else:
        # Show status of active trucks
        active = [t for t in trucks if not t.frozen]
        if active:
            print(f"Active trucks (not at freeze horizon yet):")
            for truck in sorted(active, key=lambda t: calculate_time_to_abq(t)):
                time_to_abq = calculate_time_to_abq(truck)
                print(f"    {truck.name}: {truck.position:.1f} mi & {time_to_abq:.1f} min to ABQ")

    
#update position every 5 mins...    
    for truck in trucks:
        miles_traveled = (truck.speed / 60) * 5
        truck.position -= miles_traveled
        
    time += 5


print("\n End Results:")

total_S98_cost = sum(t.actual_cost for t in trucks if t.route == 'S98')
total_I25_cost = sum(t.actual_cost for t in trucks if t.route == 'I25')

print(f"S98: {routes['S98']['count']} trucks, total cost = ${total_S98_cost:.0f}")
print(f"I25: {routes['I25']['count']} trucks, total cost = ${total_I25_cost:.0f}")
print(f"\nOverall total cost: ${total_S98_cost + total_I25_cost:.0f}")


 Time: 0 minutes
Active trucks (not at freeze horizon yet):
    Truck 19: 26.0 mi & 23.3 min to ABQ
    Truck 3: 26.0 mi & 25.2 min to ABQ
    Truck 24: 29.0 mi & 26.0 min to ABQ
    Truck 17: 27.0 mi & 26.1 min to ABQ
    Truck 4: 30.0 mi & 26.5 min to ABQ
    Truck 20: 29.0 mi & 27.6 min to ABQ
    Truck 26: 33.0 mi & 28.3 min to ABQ
    Truck 12: 25.0 mi & 28.8 min to ABQ
    Truck 2: 26.0 mi & 29.4 min to ABQ
    Truck 27: 27.0 mi & 29.5 min to ABQ
    Truck 22: 27.0 mi & 30.0 min to ABQ
    Truck 21: 38.0 mi & 33.0 min to ABQ
    Truck 9: 37.0 mi & 33.1 min to ABQ
    Truck 14: 38.0 mi & 33.5 min to ABQ
    Truck 16: 30.0 mi & 34.6 min to ABQ
    Truck 18: 40.0 mi & 35.3 min to ABQ
    Truck 23: 30.0 mi & 35.3 min to ABQ
    Truck 30: 40.0 mi & 35.3 min to ABQ
    Truck 25: 36.0 mi & 36.6 min to ABQ
    Truck 10: 42.0 mi & 38.2 min to ABQ
    Truck 1: 32.0 mi & 38.4 min to ABQ
    Truck 29: 33.0 mi & 38.8 min to ABQ
    Truck 6: 41.0 mi & 40.3 min to ABQ
    Truck 13: 36.0 mi & 4

### With BPR

In [17]:
import numpy as np

In [12]:
from dataclasses import dataclass
from typing import Optional
import random

#Create truck data class
@dataclass
class Truck:
    id: int
    name: str
    position: float  # Miles from ABQ
    speed: float     # MPH
    route: Optional[str] = None
    frozen: bool = False
    freeze_time: float = 0
    actual_cost: float = 0 


#Number of trucks? randomly 25 to 50 miles out and random speed between 50 and 70mph
def create_trucks(num_trucks):
    trucks = []
    for i in range(1, num_trucks + 1):
        truck = Truck(
            id=i,
            name=f"Truck {i}",
            position=random.randint(25, 50),  
            speed=random.randint(50, 70)     
        )
        trucks.append(truck)
    return trucks

#Time to ABQ 
def calculate_time_to_abq(truck):
    if truck.position <= 0:
        return 0
    return (truck.position / truck.speed) * 60 #in mins


def bpr_cost_function(Q, C, tf, alpha=0.15, beta=4):
    """
    Standard BPR cost function
    
    T = tf * [1 + α * (Q/C)^β]
    
    Parameters:
    - Q: traffic flow (current trucks on route)
    - C: capacity
    - tf: free-flow travel time
    - alpha: scale parameter (default 0.15)
    - beta: shape parameter (default 4)
    """
    vc_ratio = Q / C
    travel_time_bpr = tf * (1 + alpha * (vc_ratio ** beta))
    
    return travel_time_bpr
    
#assigning routes using BPR function
def assign_route(truck, routes, current_time):

    print(f"\n Evaluating routes for {truck.name}:")

    best_route = None
    best_cost = float('inf')

    for name, route in routes.items():
        Q = route['count']
        C = route['capacity']
        tf = route ['base_time']

        travel_time_bpr = bpr_cost_function(Q, C, tf)
    
        total_cost = travel_time_bpr + route['cost']
    
        # Print evaluation
        utilization = (Q / C) * 100
        print(f"    {name}:")
        print(f"      Flow: {Q}/{C} ({utilization:.0f}% capacity)")
        print(f"      Travel time: {travel_time_bpr:.1f} min")
        print(f"      Total cost: ${total_cost:.2f}")
        
        # Track best option
        if total_cost < best_cost:
            best_cost = total_cost
            best_route = name
        
    # Assign best route
    truck.route = best_route
    truck.actual_cost = best_cost
    routes[best_route]['count'] += 1
    
    print(f"Choose: {best_route} (${best_cost:.2f})")
    
    return best_cost

#Interactive
num_trucks = int(input("\nHow many trucks? "))
trucks = create_trucks(num_trucks)

# Routes with base travel times
routes = {
    'S98': {
        'cost': 7,
        'capacity': 5,
        'count': 0,
        'base_time': 32    # mins
    },
    'I25': {
        'cost': 8,
        'capacity': 5,
        'count': 0,
        'base_time': 35    
    }
}

freeze_horizon = 15.0
frozen_trucks = []


# initial positions
print("Initial positions:")
for truck in sorted(trucks, key=lambda t: t.position, reverse=True):
    time_to_abq = calculate_time_to_abq(truck)
    print(f"{truck.name}: {truck.position:.1f} mi, {truck.speed} mph & {time_to_abq:.1f} min to ABQ")


How many trucks?  30


Initial positions:
Truck 9: 46.0 mi, 59 mph & 46.8 min to ABQ
Truck 14: 46.0 mi, 55 mph & 50.2 min to ABQ
Truck 19: 46.0 mi, 61 mph & 45.2 min to ABQ
Truck 5: 44.0 mi, 59 mph & 44.7 min to ABQ
Truck 24: 44.0 mi, 70 mph & 37.7 min to ABQ
Truck 25: 44.0 mi, 51 mph & 51.8 min to ABQ
Truck 15: 42.0 mi, 54 mph & 46.7 min to ABQ
Truck 18: 42.0 mi, 62 mph & 40.6 min to ABQ
Truck 23: 42.0 mi, 53 mph & 47.5 min to ABQ
Truck 2: 41.0 mi, 52 mph & 47.3 min to ABQ
Truck 12: 41.0 mi, 54 mph & 45.6 min to ABQ
Truck 22: 41.0 mi, 62 mph & 39.7 min to ABQ
Truck 6: 38.0 mi, 70 mph & 32.6 min to ABQ
Truck 4: 37.0 mi, 59 mph & 37.6 min to ABQ
Truck 8: 37.0 mi, 50 mph & 44.4 min to ABQ
Truck 1: 35.0 mi, 52 mph & 40.4 min to ABQ
Truck 16: 33.0 mi, 56 mph & 35.4 min to ABQ
Truck 3: 32.0 mi, 60 mph & 32.0 min to ABQ
Truck 10: 32.0 mi, 56 mph & 34.3 min to ABQ
Truck 17: 32.0 mi, 50 mph & 38.4 min to ABQ
Truck 28: 32.0 mi, 60 mph & 32.0 min to ABQ
Truck 11: 31.0 mi, 54 mph & 34.4 min to ABQ
Truck 13: 29.0 mi, 61

In [14]:
# Main loop
time = 0
while any(not t.frozen for t in trucks):
    
    print(f"\n Time: {time} minutes")

    #Check for trucks at freeze horizon
    frozen_this_cycle=[]
    for truck in trucks:
        if not truck.frozen: 
            time_to_abq = calculate_time_to_abq(truck)

            if 0 < time_to_abq <= freeze_horizon:
                frozen_this_cycle.append((truck, time_to_abq))
    
    if frozen_this_cycle:
        print("Freeze Horizon Crossed")
        print(f"   {len(frozen_this_cycle)} truck(s) within 15 minutes of ABQ")
        print()
        for truck, time_to_abq in frozen_this_cycle:
            print(f"   {truck.name}: {time_to_abq:.1f} min to ABQ (≤ 15 min threshold)")
        for truck, time_to_abq in frozen_this_cycle:
                    cost = assign_route(truck, routes, time)
                    truck.frozen = True
                    truck.freeze_time = time
                    frozen_trucks.append(truck)

    else:
        # Show status of active trucks
        active = [t for t in trucks if not t.frozen]
        if active:
            print(f"Active trucks (not at freeze horizon yet):")
            for truck in sorted(active, key=lambda t: calculate_time_to_abq(t))[:3]:
                time_to_abq = calculate_time_to_abq(truck)
                print(f"    {truck.name}: {truck.position:.1f} mi & {time_to_abq:.1f} min to ABQ")

    
#update position every 5 mins...    
    for truck in trucks:
        miles_traveled = (truck.speed / 60) * 5
        truck.position -= miles_traveled
        
    time += 5


print("\n End Results:")

total_S98_cost = sum(t.actual_cost for t in trucks if t.route == 'S98')
total_I25_cost = sum(t.actual_cost for t in trucks if t.route == 'I25')

print(f"S98: {routes['S98']['count']} trucks, total cost = ${total_S98_cost:.0f}")
print(f"I25: {routes['I25']['count']} trucks, total cost = ${total_I25_cost:.0f}")
print(f"\nOverall total cost: ${total_S98_cost + total_I25_cost:.0f}")

# Show average costs
if routes['S98']['count'] > 0:
    avg_south = total_S98_cost / routes['S98']['count']
    print(f"Average cost per truck (S98): ${avg_south:.2f}")
if routes['I25']['count'] > 0:
    avg_north = total_I25_cost / routes['I25']['count']
    print(f"Average cost per truck (I25): ${avg_north:.2f}")


 Time: 0 minutes
Active trucks (not at freeze horizon yet):
    Truck 26: 27.0 mi & 23.1 min to ABQ
    Truck 30: 29.0 mi & 27.2 min to ABQ
    Truck 20: 27.0 mi & 27.9 min to ABQ

 Time: 5 minutes
Active trucks (not at freeze horizon yet):
    Truck 26: 21.2 mi & 18.1 min to ABQ
    Truck 30: 23.7 mi & 22.2 min to ABQ
    Truck 20: 22.2 mi & 22.9 min to ABQ

 Time: 10 minutes
Freeze Horizon Crossed
   1 truck(s) within 15 minutes of ABQ

   Truck 26: 13.1 min to ABQ (≤ 15 min threshold)

 Evaluating routes for Truck 26:
    S98:
      Flow: 0/5 (0% capacity)
      Travel time: 32.0 min
      Total cost: $39.00
    I25:
      Flow: 0/5 (0% capacity)
      Travel time: 35.0 min
      Total cost: $43.00
Choose: S98 ($39.00)

 Time: 15 minutes
Freeze Horizon Crossed
   5 truck(s) within 15 minutes of ABQ

   Truck 13: 13.5 min to ABQ (≤ 15 min threshold)
   Truck 20: 12.9 min to ABQ (≤ 15 min threshold)
   Truck 27: 13.4 min to ABQ (≤ 15 min threshold)
   Truck 29: 14.0 min to ABQ (≤ 15 

#### Step 2: Add trial options

#### Step 3: Add propogation

#### Step 4: Add delay distribution

Frank wolf
BPR
cost: monotonic increasing fucntion
truck demand, routing 
